# Setup

In [5]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
%pwd

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 37 (delta 13), reused 33 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 3.32 MiB | 10.42 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/kaggle/working/mlproduction


'/kaggle/working/mlproduction'

In [6]:
!pip install -q --upgrade uv
!uv sync

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 71.8 MB/s eta 0:00:00:00:0100:01
Using CPython 3.12.12
Creating virtual environment at: .venv
Resolved 34 packages in 0.98ms
Prepared 29 packages in 52.03s                                           
░░░░░░░░░░░░░░░░░░░░ [0/29] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 29 packages in 19.11s                             
 + annotated-types==0.7.0
 + filelock==3.20.0
 + fsspec==2025.12.0
 + jinja2==3.1.6
 + markupsafe==3.0.2
 + mpmath==1.3.0
 + networkx==3.6.1
 + nvidia-cublas-cu12==12.4.5.8
 + nvidia-cuda-cupti-cu12==12.4.127
 + nvidia-cuda-nvrtc-cu12==12.4.127
 + nvidia-cuda-runtime-cu12==12.

# Running

In [7]:
import torch